# 🏛️ Senior - 01. Data Governance y Calidad de Datos

**Objetivos de Aprendizaje:**
- [ ] Comprender los fundamentos de Data Governance
- [ ] Implementar frameworks de calidad de datos
- [ ] Establecer data lineage y catalogación
- [ ] Aplicar principios de DAMA-DMBOK
- [ ] Implementar validaciones automatizadas con Great Expectations
- [ ] Diseñar estrategias de metadata management

**Duración Estimada:** 150 minutos  
**Nivel de Dificultad:** Avanzado  
**Prerrequisitos:** Notebooks nivel Junior y Mid completos

---

## ⚠️ NIVEL SENIOR - PRÁCTICAS DE PRODUCCIÓN

### 🎯 A este nivel, ya sabes que:

**❌ Notebooks NO son para producción** (son solo para aprendizaje aquí)

**✅ En tu rol Senior, implementas:**
- Arquitecturas enterprise (Data Mesh, Lakehouse)
- Governance frameworks (DAMA-DMBOK)
- Pipelines de producción con Airflow/Prefect
- Infrastructure as Code (Terraform, CloudFormation)
- Observabilidad completa (OpenTelemetry, Datadog)
- Security & Compliance (GDPR, SOC2)

**📖 Referencia obligatoria:** `notebooks/⚠️_IMPORTANTE_LEER_PRIMERO.md`

**Este nivel te enseña:** Diseño de arquitecturas y governanza, luego lo implementas en código de producción.

---

**Autor:** LuisRai (Luis J. Raigoso V.) | © 2024-2025

---

### 🏛️ **Data Governance: El Fundamento de Datos Confiables**

**¿Por qué Data Governance es crítico en Senior-level?**

A nivel Senior, ya no solo implementas pipelines técnicos → **diseñas la estrategia organizacional** de cómo los datos se gestionan, gobiernan y monetizan.

**Evolución del Data Engineer:**

```
Junior:  "¿Cómo muevo datos de A a B?"
Mid:     "¿Cómo lo hago escalable, confiable y monitoreado?"
Senior:  "¿Cómo aseguro que TODOS los datos de la org sean confiables, 
          cumplan regulaciones y generen valor de negocio?"
```

**Impacto de Mala Calidad de Datos:**

| Sector | Costo Anual (Industria) | Ejemplo Real |
|--------|-------------------------|--------------|
| **Retail** | $3.1 trillones USD | Target: Promoción a clientes equivocados → $1M perdido |
| **Healthcare** | $74B USD | Registros duplicados → medicación incorrecta |
| **Finance** | $15M/empresa | Knight Capital: Bug en prod → $440M en 45 min |
| **Manufacturing** | $5M/empresa | Datos de inventario desactualizados → Stockouts |

**Data Governance Framework (Componentes):**

```
┌────────────────────────────────────────────────────────┐
│                    GOVERNANCE LAYER                     │
│                                                         │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐            │
│  │  Policy  │  │  Roles   │  │Standards │            │
│  │ Políticas│  │ Stewards │  │Estándares│            │
│  └──────────┘  └──────────┘  └──────────┘            │
└──────────────────────┬─────────────────────────────────┘
                       │
       ┌───────────────┼───────────────┐
       ▼               ▼               ▼
┌──────────┐    ┌──────────┐    ┌──────────┐
│  Data    │    │   Data   │    │   Data   │
│ Quality  │    │ Security │    │ Lineage  │
│          │    │          │    │          │
│ • 6 dim. │    │ • RBAC   │    │ • Origen │
│ • SLAs   │    │ • Encrypt│    │ • Impacto│
│ • Monitor│    │ • Audit  │    │ • Deps   │
└──────────┘    └──────────┘    └──────────┘
       │               │               │
       └───────────────┴───────────────┘
                       │
                       ▼
            ┌──────────────────┐
            │  Data Catalog    │
            │  (Metadata Hub)  │
            │                  │
            │  • Assets        │
            │  • Ownership     │
            │  • Glossary      │
            └──────────────────┘
```

**DAMA-DMBOK (Data Management Body of Knowledge):**

Framework estándar de la industria con 11 áreas:

1. **Data Governance** (Core): Marco organizacional
2. **Data Architecture**: Blueprints de sistemas de datos
3. **Data Modeling**: Diseño de estructuras
4. **Data Storage & Operations**: Almacenamiento
5. **Data Security**: Protección y privacidad
6. **Data Integration & Interoperability**: ETL/ELT
7. **Document & Content Management**: Docs no estructurados
8. **Reference & Master Data**: Golden records (MDM)
9. **Data Warehousing & BI**: Analytics
10. **Metadata Management**: "Data about data"
11. **Data Quality**: 6 dimensiones críticas

**Roles en Data Governance:**

```python
class DataGovernanceTeam:
    """
    Chief Data Officer (CDO)
      ├── Data Governance Manager
      │     ├── Data Stewards (por dominio: Ventas, Finanzas, HR)
      │     └── Data Quality Analysts
      ├── Data Architect
      ├── Data Security Officer
      └── Compliance Officer
    """
    
    roles = {
        'Data Steward': 'Define reglas de negocio, resuelve issues de calidad',
        'Data Owner': 'Responsable final de un dominio de datos',
        'Data Custodian': 'Implementa controles técnicos (Data Engineer)',
        'Data Consumer': 'Usa datos para análisis/decisiones'
    }
```

**Herramientas del Ecosistema:**

| Categoría | Open Source | Comercial |
|-----------|-------------|-----------|
| **Data Catalog** | Apache Atlas, Amundsen, DataHub | Alation, Collibra, Informatica |
| **Data Quality** | Great Expectations, Soda, dbt tests | Talend DQ, Informatica DQ |
| **Data Lineage** | OpenLineage, Marquez | Manta, Collibra Lineage |
| **MDM (Master Data)** | - | Informatica MDM, SAP MDM |
| **Metadata Mgmt** | Apache Atlas | Collibra, Alation |

**Métricas de Data Governance (KPIs):**

```python
governance_metrics = {
    'Data Quality Score': '% de datasets que pasan quality gates',
    'Metadata Coverage': '% de tablas con descripciones/owners',
    'Policy Compliance': '% de datasets que cumplen políticas',
    'Mean Time to Remediation': 'Tiempo promedio para arreglar issues',
    'Data Lineage Coverage': '% de pipelines con lineage documentado',
    'Security Incidents': '# de breaches/accesos no autorizados'
}

# Ejemplo de SLA
quality_sla = {
    'Critical datasets': {
        'completeness': '> 99%',
        'freshness': '< 1 hour',
        'accuracy': '> 99.5%'
    },
    'Standard datasets': {
        'completeness': '> 95%',
        'freshness': '< 24 hours',
        'accuracy': '> 95%'
    }
}
```

**Regulaciones de Compliance:**

- **GDPR** (Europa): Derecho al olvido, portabilidad, consentimiento
- **CCPA** (California): Privacidad de consumidores
- **HIPAA** (USA): Datos de salud
- **SOX** (USA): Datos financieros
- **LGPD** (Brasil): Protección de datos personales

**Implementación Práctica en Pipelines:**

```python
# Pipeline con governance embebido
@data_quality_check(min_score=95)
@pii_detector(action='mask')
@lineage_tracking(catalog='atlas')
def ingest_customer_data(source, dest):
    df = extract(source)
    df = validate_schema(df)          # Quality
    df = mask_pii_columns(df)         # Security
    df = enrich_metadata(df)          # Catalog
    load(df, dest)
    track_lineage(source, dest)       # Lineage
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 🎯 ¿Qué es Data Governance?

**Data Governance** es el conjunto de procesos, políticas, estándares y métricas que aseguran el uso efectivo y eficiente de la información, habilitando a una organización a alcanzar sus objetivos.

### 🏗️ Pilares de Data Governance:

1. **Data Quality** (Calidad de Datos)
   - Precisión, completitud, consistencia
   - Validaciones y monitoreo
   - Remediación de problemas

2. **Data Security** (Seguridad)
   - Control de accesos
   - Encriptación
   - Auditoría

3. **Data Lineage** (Linaje)
   - Trazabilidad origen-destino
   - Impacto de cambios
   - Documentación automática

4. **Data Catalog** (Catálogo)
   - Inventario de activos de datos
   - Metadata management
   - Descubrimiento y búsqueda

5. **Compliance** (Cumplimiento)
   - GDPR, CCPA, HIPAA
   - Políticas de retención
   - Auditorías regulatorias

### 🌟 Beneficios:

```
✅ Confianza en los datos
✅ Reducción de riesgos
✅ Cumplimiento regulatorio
✅ Mejor toma de decisiones
✅ Eficiencia operacional
✅ Colaboración mejorada
```

### 📊 **Data Quality: Las 6 Dimensiones Críticas (ISO 8000)**

**Framework de Calidad basado en estándar ISO 8000:**

**1. COMPLETITUD (Completeness):**

```python
# ¿Todos los datos esperados están presentes?

# Ejemplo: Dataset de clientes
df_clientes = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'nombre': ['Ana', 'Bob', None, 'Diana', 'Eva'],  # 1 null
    'email': ['a@x.com', None, 'c@x.com', 'd@x.com', None],  # 2 nulls
    'telefono': ['111', '222', '333', None, '555']  # 1 null
})

# Métrica: % de valores no-null
completitud = (1 - df_clientes.isnull().sum().sum() / df_clientes.size) * 100
# = (1 - 4 / 20) * 100 = 80%

# SLA típico: > 95% para campos críticos, > 85% para opcionales
```

**Causas de incompletitud:**
- Campos opcionales en formularios
- Errores en integración de sistemas
- Cambios de schema sin backfill

**Remediación:**
```python
# Imputación inteligente
df['telefono'] = df['telefono'].fillna('UNKNOWN')

# Alertas automáticas
if completitud < 95:
    send_slack_alert(f"Completitud baja: {completitud}%")
```

---

**2. UNICIDAD (Uniqueness):**

```python
# ¿Los registros son únicos según business key?

# Ejemplo: Duplicados por error de ETL
df_ventas = pd.DataFrame({
    'venta_id': [1, 2, 2, 3],  # venta_id=2 duplicado
    'cliente': ['A', 'B', 'B', 'C'],
    'total': [100, 200, 200, 150]
})

# Métrica
duplicados = df_ventas.duplicated(subset=['venta_id']).sum()  # 1
unicidad = (1 - duplicados / len(df_ventas)) * 100  # 75%

# Identificar duplicados
df_ventas[df_ventas.duplicated(subset=['venta_id'], keep=False)]
```

**Causas de duplicados:**
- Retry logic sin idempotencia
- Merge de múltiples fuentes sin deduplicación
- Race conditions en escrituras concurrentes

**Remediación:**
```python
# Deduplicación con estrategia
df_clean = df_ventas.drop_duplicates(subset=['venta_id'], keep='last')

# O upsert en DB
INSERT INTO ventas (...) VALUES (...)
ON CONFLICT (venta_id) DO UPDATE SET ...
```

---

**3. VALIDEZ (Validity):**

```python
# ¿Los valores cumplen reglas de negocio?

# Reglas de negocio
reglas = {
    'cantidad_positiva': 'cantidad > 0',
    'precio_razonable': 'precio_unitario >= 0 and precio_unitario <= 10000',
    'descuento_valido': 'descuento_pct >= 0 and descuento_pct <= 100',
    'total_coherente': 'total == cantidad * precio_unitario * (1 - descuento_pct/100)'
}

# Validación
for regla, condicion in reglas.items():
    violaciones = (~df.eval(condicion)).sum()
    if violaciones > 0:
        print(f"⚠️ {regla}: {violaciones} violaciones")

# Ejemplo de violaciones
# cantidad: -5 (negativa) → Inválida
# descuento_pct: 150% → Inválida
```

**Tipos de validaciones:**
- **Tipo de dato**: `isinstance(valor, int)`
- **Rango**: `0 <= edad <= 120`
- **Formato**: `email.match(r'^\S+@\S+\.\S+$')`
- **Referencial**: `cliente_id in clientes.id`
- **Lógica de negocio**: `fecha_entrega > fecha_pedido`

---

**4. CONSISTENCIA (Consistency):**

```python
# ¿Los datos son coherentes entre sí y a lo largo del tiempo?

# Inconsistencia cross-field
df['total_calculado'] = df['cantidad'] * df['precio_unitario']
df['inconsistencia'] = abs(df['total'] - df['total_calculado']) > 0.01

# Inconsistencia temporal
df['mes_factura'] = pd.to_datetime(df['fecha_factura']).dt.month
df['mes_pago'] = pd.to_datetime(df['fecha_pago']).dt.month
inconsistentes = df[df['mes_pago'] < df['mes_factura']]  # Pago antes de factura

# Inconsistencia entre sistemas
df_erp['stock'] != df_warehouse['stock']  # Desincronización
```

**Causas:**
- Falta de transacciones ACID
- Sincronización eventual entre sistemas
- Cambios de lógica de negocio

---

**5. EXACTITUD (Accuracy):**

```python
# ¿Los valores reflejan la realidad?

# Ejemplo: Comparar con fuente de verdad (ground truth)
df_measured = pd.DataFrame({'temp': [20.1, 25.3, 30.5]})
df_ground_truth = pd.DataFrame({'temp': [20.0, 25.0, 30.0]})

# RMSE (Root Mean Squared Error)
rmse = np.sqrt(((df_measured['temp'] - df_ground_truth['temp'])**2).mean())
# RMSE = 0.36°C

# Exactitud
exactitud = 100 - (rmse / df_ground_truth['temp'].mean()) * 100
```

**Difícil de medir sin "ground truth":**
- Requiere auditorías manuales
- Sampling aleatorio + verificación humana
- Comparación con fuentes externas confiables

---

**6. ACTUALIDAD (Timeliness/Freshness):**

```python
# ¿Los datos son suficientemente recientes?

# Ejemplo
hoy = pd.Timestamp.now()
df['lag'] = (hoy - pd.to_datetime(df['fecha_ingestion'])).dt.total_seconds() / 3600

# SLA: Datos críticos < 1 hora
df_criticos = df[df['categoria'] == 'critical']
sla_breach = df_criticos[df_criticos['lag'] > 1]

print(f"SLA breaches: {len(sla_breach)} datasets")
```

**SLAs típicos:**
- **Real-time**: < 1 minuto (fraud detection)
- **Near real-time**: < 15 minutos (operational dashboards)
- **Batch**: < 24 horas (reporting)
- **Archive**: > 7 días (historical analysis)

---

**Data Quality Score Agregado:**

```python
def calculate_dq_score(df, rules):
    """Score compuesto 0-100"""
    
    scores = {
        'completitud': (1 - df.isnull().sum().sum() / df.size) * 100,
        'unicidad': (1 - df.duplicated().sum() / len(df)) * 100,
        'validez': calcular_validez(df, rules),
        'consistencia': calcular_consistencia(df),
        'exactitud': 95,  # Requiere ground truth
        'actualidad': calcular_freshness(df, 'fecha')
    }
    
    # Weighted average (ponderado según criticidad)
    weights = {
        'completitud': 0.25,
        'unicidad': 0.20,
        'validez': 0.25,
        'consistencia': 0.15,
        'exactitud': 0.10,
        'actualidad': 0.05
    }
    
    dq_score = sum(scores[k] * weights[k] for k in scores)
    
    return dq_score, scores
```

**Niveles de Calidad:**
- **⭐⭐⭐ Excelente**: 95-100% → Prod-ready
- **⭐⭐ Bueno**: 85-94% → Requiere atención
- **⭐ Aceptable**: 70-84% → Plan de remediación
- **❌ Crítico**: < 70% → Bloquear uso

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 📊 Framework DAMA-DMBOK

### Las 11 Áreas de Conocimiento:

1. **Data Governance** - Marco general
2. **Data Architecture** - Estructura y diseño
3. **Data Modeling & Design** - Modelado de datos
4. **Data Storage & Operations** - Almacenamiento y operaciones
5. **Data Security** - Seguridad
6. **Data Integration & Interoperability** - Integración
7. **Document & Content Management** - Gestión documental
8. **Reference & Master Data** - Datos maestros
9. **Data Warehousing & BI** - Almacenes y BI
10. **Metadata Management** - Gestión de metadatos
11. **Data Quality** - Calidad de datos

In [ ]:
# Imports necesarios
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import hashlib
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas")
print(f"📅 Sesión iniciada: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 🔍 Dimensiones de Calidad de Datos

### Las 6 Dimensiones Críticas:

### 🛠️ **Data Quality Framework: Implementación Productiva**

**¿Por qué un Framework y no validaciones ad-hoc?**

```python
# ❌ Enfoque ad-hoc (no escalable)
if df['precio'].isnull().sum() > 0:
    print("Hay nulls en precio")
if len(df[df['cantidad'] < 0]) > 0:
    print("Cantidades negativas")
# ... 50 validaciones más dispersas en el código

# ✅ Enfoque Framework (escalable, reusable)
dq_framework = DataQualityFramework(df, "ventas")
dq_framework.evaluar_completitud()
dq_framework.evaluar_validez(business_rules)
dq_framework.generar_reporte_completo()
```

**Componentes del Framework:**

```
DataQualityFramework
├── __init__()              → Inicialización con dataset
├── evaluar_completitud()   → Dimensión 1
├── evaluar_unicidad()      → Dimensión 2
├── evaluar_validez()       → Dimensión 3
├── evaluar_consistencia()  → Dimensión 4
├── evaluar_exactitud()     → Dimensión 5
├── evaluar_actualidad()    → Dimensión 6
├── generar_reporte()       → JSON output
└── _clasificar_calidad()   → Score → Rating
```

**Arquitectura de Calidad en Pipelines:**

```
┌──────────────────────────────────────────┐
│         SOURCE SYSTEM (API/DB)           │
└─────────────────┬────────────────────────┘
                  │
                  ▼
       ┌──────────────────┐
       │   RAW LAYER      │  ← Landing zone (sin validar)
       └────────┬─────────┘
                │
                ▼
       ┌──────────────────────────┐
       │  DATA QUALITY CHECKS     │
       │  (DataQualityFramework)  │
       │                          │
       │  IF score < threshold:   │
       │    → DLQ (Dead Letter)   │
       │    → Alert team          │
       │    → Block downstream    │
       │  ELSE:                   │
       │    → Continue pipeline   │
       └────────┬─────────────────┘
                │
                ▼
       ┌──────────────────┐
       │  TRUSTED LAYER   │  ← Datos validados
       └────────┬─────────┘
                │
                ▼
       ┌──────────────────┐
       │ REFINED LAYER    │  ← Datos refinados
       └──────────────────┘
```

**Integration con Airflow:**

```python
from airflow import DAG
from airflow.operators.python import PythonOperator, BranchPythonOperator

def check_data_quality(**context):
    df = context['ti'].xcom_pull(task_ids='extract_data')
    
    dq = DataQualityFramework(df, "ventas_daily")
    dq.evaluar_completitud()
    dq.evaluar_unicidad(['venta_id'])
    dq.evaluar_validez(business_rules)
    
    report = dq.generar_reporte_completo()
    context['ti'].xcom_push(key='dq_score', value=report['score_general'])
    
    return 'pass_quality' if report['score_general'] >= 90 else 'fail_quality'

with DAG('pipeline_with_dq', ...) as dag:
    extract = PythonOperator(task_id='extract_data', ...)
    
    quality_check = BranchPythonOperator(
        task_id='quality_check',
        python_callable=check_data_quality
    )
    
    pass_quality = PythonOperator(task_id='pass_quality', ...)
    fail_quality = PythonOperator(task_id='fail_quality', ...)  # Send alert
    
    extract >> quality_check >> [pass_quality, fail_quality]
```

**Great Expectations Integration (Alternative):**

```python
import great_expectations as gx

# Setup
context = gx.get_context()
suite = context.create_expectation_suite("ventas_suite")

# Expectations (equivalente a nuestras dimensiones)
validator = context.sources.pandas_default.read_dataframe(df)

# Completitud
validator.expect_column_values_to_not_be_null("precio")

# Unicidad
validator.expect_column_values_to_be_unique("venta_id")

# Validez (rango)
validator.expect_column_values_to_be_between("cantidad", min_value=0, max_value=1000)

# Consistencia
validator.expect_column_pair_values_A_to_be_greater_than_B(
    column_A="fecha_entrega", 
    column_B="fecha_pedido"
)

# Run validation
results = validator.validate()
print(results.success)  # True/False
```

**Comparación de Herramientas:**

| Feature | Custom Framework | Great Expectations | dbt tests |
|---------|------------------|-------------------|-----------|
| **Setup** | Simple (1 clase) | Complejo (contexto, suite) | Medio (SQL) |
| **Flexibilidad** | Alta | Media (extensible) | Media |
| **Visualización** | DIY | Data Docs automático | dbt Cloud |
| **Integración** | Manual | Airflow/Prefect | dbt native |
| **Learning Curve** | Bajo | Alto | Medio |
| **Best For** | Custom rules | Enterprise governance | SQL-centric teams |

**Logging y Observability:**

```python
import logging
from pythonjsonlogger import jsonlogger

# Structured logging
logger = logging.getLogger(__name__)
handler = logging.StreamHandler()
formatter = jsonlogger.JsonFormatter()
handler.setFormatter(formatter)
logger.addHandler(handler)

def evaluar_completitud(self):
    logger.info("DQ check started", extra={
        'check_type': 'completitud',
        'dataset': self.nombre_dataset,
        'total_records': len(self.df)
    })
    
    # ... evaluación ...
    
    logger.info("DQ check completed", extra={
        'check_type': 'completitud',
        'score': completitud_pct,
        'passed': completitud_pct >= 95
    })
    
    return completitud_pct
```

**Métricas para Prometheus:**

```python
from prometheus_client import Counter, Histogram, Gauge

dq_checks_total = Counter('dq_checks_total', 'Total DQ checks', ['dataset', 'dimension'])
dq_score = Gauge('dq_score', 'DQ score 0-100', ['dataset'])
dq_violations = Counter('dq_violations_total', 'Total violations', ['dataset', 'rule'])

def evaluar_completitud(self):
    dq_checks_total.labels(dataset=self.nombre_dataset, dimension='completitud').inc()
    
    # ... evaluación ...
    
    dq_score.labels(dataset=self.nombre_dataset).set(completitud_pct)
    
    if completitud_pct < 95:
        dq_violations.labels(dataset=self.nombre_dataset, rule='completitud_threshold').inc()
```

**Testing del Framework:**

```python
import pytest

def test_completitud_100_percent():
    df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
    dq = DataQualityFramework(df, "test")
    score = dq.evaluar_completitud()
    assert score == 100.0

def test_unicidad_detecta_duplicados():
    df = pd.DataFrame({'id': [1, 2, 2, 3]})
    dq = DataQualityFramework(df, "test")
    score = dq.evaluar_unicidad(['id'])
    assert score == 75.0  # 3/4 únicos

@pytest.mark.parametrize("cantidad,expected_valid", [
    (5, True),
    (-1, False),
    (0, False)
])
def test_validez_cantidad_positiva(cantidad, expected_valid):
    df = pd.DataFrame({'cantidad': [cantidad]})
    dq = DataQualityFramework(df, "test")
    score = dq.evaluar_validez({'cantidad_positiva': 'cantidad > 0'})
    assert (score == 100.0) == expected_valid
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
class DataQualityFramework:
    """
    Framework para evaluar calidad de datos
    """
    
    def __init__(self, df: pd.DataFrame, nombre_dataset: str):
        self.df = df
        self.nombre_dataset = nombre_dataset
        self.reporte = {
            'dataset': nombre_dataset,
            'fecha_evaluacion': datetime.now().isoformat(),
            'total_registros': len(df),
            'total_columnas': len(df.columns),
            'dimensiones': {}
        }
    
    def evaluar_completitud(self):
        """
        Dimensión 1: Completitud
        ¿Los datos están completos?
        """
        print("📊 Evaluando COMPLETITUD...")
        
        total_valores = self.df.size
        valores_nulos = self.df.isnull().sum().sum()
        completitud_pct = ((total_valores - valores_nulos) / total_valores) * 100
        
        # Por columna
        completitud_cols = {}
        for col in self.df.columns:
            pct = (1 - self.df[col].isnull().sum() / len(self.df)) * 100
            completitud_cols[col] = round(pct, 2)
        
        self.reporte['dimensiones']['completitud'] = {
            'score': round(completitud_pct, 2),
            'valores_nulos': int(valores_nulos),
            'por_columna': completitud_cols,
            'aprobado': completitud_pct >= 95
        }
        
        print(f"   ✓ Score: {completitud_pct:.2f}%")
        print(f"   ✓ Valores nulos: {valores_nulos:,}")
        return completitud_pct
    
    def evaluar_unicidad(self, columnas_clave: list):
        """
        Dimensión 2: Unicidad
        ¿Los registros son únicos?
        """
        print("\n🔑 Evaluando UNICIDAD...")
        
        duplicados = self.df.duplicated(subset=columnas_clave).sum()
        unicidad_pct = ((len(self.df) - duplicados) / len(self.df)) * 100
        
        self.reporte['dimensiones']['unicidad'] = {
            'score': round(unicidad_pct, 2),
            'duplicados': int(duplicados),
            'columnas_evaluadas': columnas_clave,
            'aprobado': duplicados == 0
        }
        
        print(f"   ✓ Score: {unicidad_pct:.2f}%")
        print(f"   ✓ Duplicados: {duplicados}")
        return unicidad_pct
    
    def evaluar_validez(self, reglas: dict):
        """
        Dimensión 3: Validez
        ¿Los datos cumplen reglas de negocio?
        """
        print("\n✅ Evaluando VALIDEZ...")
        
        violaciones = {}
        total_registros = len(self.df)
        registros_validos = total_registros
        
        for regla_nombre, regla_condicion in reglas.items():
            try:
                registros_invalidos = (~self.df.eval(regla_condicion)).sum()
                violaciones[regla_nombre] = int(registros_invalidos)
                registros_validos -= registros_invalidos
                print(f"   • {regla_nombre}: {registros_invalidos} violaciones")
            except Exception as e:
                print(f"   ⚠️ Error en regla '{regla_nombre}': {e}")
        
        validez_pct = (registros_validos / total_registros) * 100
        
        self.reporte['dimensiones']['validez'] = {
            'score': round(validez_pct, 2),
            'violaciones': violaciones,
            'reglas_evaluadas': list(reglas.keys()),
            'aprobado': validez_pct >= 95
        }
        
        print(f"   ✓ Score: {validez_pct:.2f}%")
        return validez_pct
    
    def evaluar_consistencia(self, columnas_relacionadas: dict):
        """
        Dimensión 4: Consistencia
        ¿Los datos son consistentes entre sí?
        """
        print("\n🔄 Evaluando CONSISTENCIA...")
        
        inconsistencias = {}
        
        for nombre, condicion in columnas_relacionadas.items():
            try:
                count = (~self.df.eval(condicion)).sum()
                inconsistencias[nombre] = int(count)
                print(f"   • {nombre}: {count} inconsistencias")
            except Exception as e:
                print(f"   ⚠️ Error en validación '{nombre}': {e}")
        
        total_inconsistencias = sum(inconsistencias.values())
        consistencia_pct = ((len(self.df) - total_inconsistencias) / len(self.df)) * 100
        
        self.reporte['dimensiones']['consistencia'] = {
            'score': round(consistencia_pct, 2),
            'inconsistencias': inconsistencias,
            'aprobado': consistencia_pct >= 95
        }
        
        print(f"   ✓ Score: {consistencia_pct:.2f}%")
        return consistencia_pct
    
    def evaluar_exactitud(self, columna_numerica: str, rango_esperado: tuple):
        """
        Dimensión 5: Exactitud
        ¿Los valores son correctos?
        """
        print("\n🎯 Evaluando EXACTITUD...")
        
        min_val, max_val = rango_esperado
        fuera_rango = ((self.df[columna_numerica] < min_val) | 
                       (self.df[columna_numerica] > max_val)).sum()
        
        exactitud_pct = ((len(self.df) - fuera_rango) / len(self.df)) * 100
        
        self.reporte['dimensiones']['exactitud'] = {
            'score': round(exactitud_pct, 2),
            'fuera_de_rango': int(fuera_rango),
            'columna_evaluada': columna_numerica,
            'rango_esperado': rango_esperado,
            'aprobado': fuera_rango == 0
        }
        
        print(f"   ✓ Score: {exactitud_pct:.2f}%")
        print(f"   ✓ Fuera de rango: {fuera_rango}")
        return exactitud_pct
    
    def evaluar_actualidad(self, columna_fecha: str, dias_maximos: int):
        """
        Dimensión 6: Actualidad
        ¿Los datos son recientes?
        """
        print("\n📅 Evaluando ACTUALIDAD...")
        
        hoy = pd.Timestamp.now()
        self.df[columna_fecha] = pd.to_datetime(self.df[columna_fecha])
        antiguos = (hoy - self.df[columna_fecha]) > timedelta(days=dias_maximos)
        registros_antiguos = antiguos.sum()
        
        actualidad_pct = ((len(self.df) - registros_antiguos) / len(self.df)) * 100
        
        self.reporte['dimensiones']['actualidad'] = {
            'score': round(actualidad_pct, 2),
            'registros_antiguos': int(registros_antiguos),
            'dias_maximos': dias_maximos,
            'aprobado': actualidad_pct >= 80
        }
        
        print(f"   ✓ Score: {actualidad_pct:.2f}%")
        print(f"   ✓ Registros antiguos: {registros_antiguos}")
        return actualidad_pct
    
    def generar_reporte_completo(self):
        """
        Genera reporte ejecutivo de calidad
        """
        print("\n" + "="*60)
        print(f"📊 REPORTE DE CALIDAD DE DATOS: {self.nombre_dataset}")
        print("="*60)
        
        # Calcular score general
        scores = [d['score'] for d in self.reporte['dimensiones'].values() if 'score' in d]
        score_general = sum(scores) / len(scores) if scores else 0
        
        self.reporte['score_general'] = round(score_general, 2)
        self.reporte['nivel_calidad'] = self._clasificar_calidad(score_general)
        
        print(f"\n🎯 Score General: {score_general:.2f}%")
        print(f"📈 Nivel de Calidad: {self.reporte['nivel_calidad']}")
        print(f"\n📋 Detalles por Dimensión:")
        
        for dimension, datos in self.reporte['dimensiones'].items():
            status = "✅" if datos.get('aprobado', False) else "❌"
            print(f"   {status} {dimension.upper()}: {datos.get('score', 'N/A')}%")
        
        # Guardar reporte
        output_path = f"../datasets/processed/reporte_calidad_{self.nombre_dataset}.json"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(self.reporte, f, indent=2, ensure_ascii=False)
        
        print(f"\n💾 Reporte guardado en: {output_path}")
        return self.reporte
    
    @staticmethod
    def _clasificar_calidad(score):
        """Clasifica el nivel de calidad según el score"""
        if score >= 95:
            return "⭐⭐⭐ EXCELENTE"
        elif score >= 85:
            return "⭐⭐ BUENO"
        elif score >= 70:
            return "⭐ ACEPTABLE"
        else:
            return "❌ CRÍTICO"

print("✅ DataQualityFramework definido")

## 🧪 Caso Práctico: Evaluación de Calidad

Vamos a evaluar un dataset de ventas:

### 🧪 **Caso Práctico: From Bad Data to Gold Standard**

**Escenario Real:**

Empresa de e-commerce con 1M+ ventas diarias. El equipo de Analytics reporta:
- ❌ Dashboards muestran ventas negativas
- ❌ Duplicados causan sobreconteo de revenue
- ❌ Missing data en ~20% de registros
- ❌ Descuentos > 100% generan alertas falsas

**Impacto:** CFO no confía en reportes → Decisiones basadas en "feeling"

**Solución: Implementar DQ Framework**

**Step 1: Assessment (Baseline)**

```python
# Snapshot inicial
dq_initial = DataQualityFramework(df_ventas_raw, "ventas_oct_2025")

# Run all dimensions
dq_initial.evaluar_completitud()        # 80%  ❌
dq_initial.evaluar_unicidad(['venta_id'])  # 99%  ⚠️ (10 duplicados)
dq_initial.evaluar_validez(business_rules)  # 75%  ❌
dq_initial.evaluar_consistencia({...})   # 85%  ⚠️

report_initial = dq_initial.generar_reporte_completo()
# Score general: 82% → ⭐ ACEPTABLE (pero inaceptable para prod)
```

**Step 2: Data Profiling (Root Cause Analysis)**

```python
# ¿Qué columnas tienen más problemas?
completitud_por_columna = report_initial['dimensiones']['completitud']['por_columna']
worst_columns = sorted(completitud_por_columna.items(), key=lambda x: x[1])[:5]

print("Top 5 columnas con más nulls:")
for col, pct in worst_columns:
    print(f"  - {col}: {100-pct:.1f}% nulls")
    
# Output:
#   - producto: 15% nulls         → API devuelve null cuando SKU inválido
#   - descuento_pct: 10% nulls    → Campo opcional en checkout
#   - email_cliente: 8% nulls     → Guests no proveen email
#   - fecha_entrega: 20% nulls    → Solo para órdenes enviadas
#   - cupon_id: 60% nulls         → Mayoría de ventas sin cupón
```

**Step 3: Remediation Plan**

```python
# Plan de remediación priorizado
remediation_plan = {
    'P0_Critical': [
        {
            'issue': 'Cantidades negativas',
            'rule': 'cantidad > 0',
            'fix': 'Rechazar en validación pre-insert',
            'owner': 'Backend team'
        },
        {
            'issue': 'Duplicados por retry',
            'rule': 'venta_id unique',
            'fix': 'Implement idempotency key',
            'owner': 'Platform team'
        }
    ],
    'P1_High': [
        {
            'issue': 'Missing producto',
            'rule': 'producto not null',
            'fix': 'Lookup SKU en master data antes de insert',
            'owner': 'Data team'
        },
        {
            'issue': 'Descuentos > 100%',
            'rule': 'descuento_pct between 0 and 100',
            'fix': 'Validación en frontend + backend',
            'owner': 'Frontend team'
        }
    ],
    'P2_Medium': [
        {
            'issue': 'Email null',
            'rule': 'email not null for registered users',
            'fix': 'Default to "guest@example.com" for guests',
            'owner': 'Data team'
        }
    ]
}
```

**Step 4: Implementation (Data Cleaning)**

```python
def clean_ventas_data(df_raw):
    """Pipeline de limpieza aplicando remediaciones"""
    
    df = df_raw.copy()
    
    # Fix 1: Remover cantidades negativas (P0)
    before = len(df)
    df = df[df['cantidad'] > 0]
    logger.info(f"Removed {before - len(df)} records with cantidad <= 0")
    
    # Fix 2: Deduplicar por venta_id (P0)
    df = df.drop_duplicates(subset=['venta_id'], keep='last')
    logger.info(f"Removed {before - len(df)} duplicates")
    
    # Fix 3: Imputar producto missing (P1)
    df.loc[df['producto'].isnull(), 'producto'] = 'UNKNOWN'
    
    # Fix 4: Caps descuentos (P1)
    df['descuento_pct'] = df['descuento_pct'].clip(lower=0, upper=100)
    
    # Fix 5: Imputar email (P2)
    df.loc[df['email_cliente'].isnull() & df['tipo_cliente'] == 'guest', 
           'email_cliente'] = 'guest@example.com'
    
    # Fix 6: Recalcular total para consistencia
    df['total'] = (df['cantidad'] * df['precio_unitario'] * 
                   (1 - df['descuento_pct'] / 100))
    
    return df

df_ventas_clean = clean_ventas_data(df_ventas_raw)
```

**Step 5: Re-assessment (Validation)**

```python
# Re-evaluar post-limpieza
dq_final = DataQualityFramework(df_ventas_clean, "ventas_oct_2025_clean")

dq_final.evaluar_completitud()        # 98%  ✅
dq_final.evaluar_unicidad(['venta_id'])  # 100%  ✅
dq_final.evaluar_validez(business_rules)  # 99%  ✅
dq_final.evaluar_consistencia({...})   # 99%  ✅

report_final = dq_final.generar_reporte_completo()
# Score general: 97.5% → ⭐⭐⭐ EXCELENTE
```

**Step 6: Monitoring (Continuous)**

```python
# Airflow DAG diario
@dag(schedule_interval='@daily')
def ventas_dq_monitoring():
    
    @task
    def extract_ventas():
        return query_db("SELECT * FROM ventas WHERE date = CURRENT_DATE")
    
    @task
    def run_dq_checks(df):
        dq = DataQualityFramework(df, f"ventas_{datetime.now().date()}")
        report = dq.generar_reporte_completo()
        
        # Store metrics in TimeSeries DB
        store_metrics(report)
        
        # Alert if regression
        if report['score_general'] < 95:
            send_slack_alert(
                channel="#data-quality",
                message=f"⚠️ DQ score dropped to {report['score_general']}%",
                report_url=f"/reports/{report['dataset']}"
            )
        
        return report
    
    df = extract_ventas()
    run_dq_checks(df)

dag = ventas_dq_monitoring()
```

**Results:**

| Metric | Before | After | Improvement |
|--------|--------|-------|-------------|
| **DQ Score** | 82% | 97.5% | +15.5% |
| **Duplicates** | 10/day | 0 | 100% |
| **Completitud** | 80% | 98% | +18% |
| **Validez** | 75% | 99% | +24% |
| **CFO Confidence** | ❌ | ✅ | Priceless |

**Business Impact:**
- ✅ Revenue reporting accuracy: +$500K/month (false negatives eliminados)
- ✅ Compliance audit: Pass (antes Fail)
- ✅ Data analyst productivity: +30% (menos tiempo limpiando)
- ✅ Executive trust: "Now I can make data-driven decisions"

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
# Crear dataset de ejemplo con problemas de calidad intencionales
np.random.seed(42)

n_registros = 1000

df_ventas = pd.DataFrame({
    'venta_id': range(1, n_registros + 1),
    'cliente_id': np.random.randint(1, 200, n_registros),
    'producto': np.random.choice(['Laptop', 'Mouse', 'Teclado', 'Monitor', None], n_registros),
    'cantidad': np.random.randint(-5, 50, n_registros),  # Algunas cantidades negativas
    'precio_unitario': np.random.uniform(10, 2000, n_registros),
    'descuento_pct': np.random.uniform(0, 100, n_registros),  # Algunos > 100%
    'total': np.random.uniform(0, 5000, n_registros),
    'fecha_venta': pd.date_range(start='2023-01-01', periods=n_registros, freq='H'),
    'estado': np.random.choice(['Completada', 'Pendiente', 'Cancelada', None], n_registros)
})

# Introducir duplicados intencionales
df_ventas = pd.concat([df_ventas, df_ventas.iloc[:10]], ignore_index=True)

# Introducir algunos nulls adicionales
df_ventas.loc[np.random.choice(df_ventas.index, 50), 'precio_unitario'] = np.nan

print(f"📊 Dataset creado: {len(df_ventas)} registros, {len(df_ventas.columns)} columnas")
print(f"\n🔍 Primeras filas:")
print(df_ventas.head())
print(f"\n📈 Info del dataset:")
print(df_ventas.info())

In [ ]:
# Inicializar framework
dq = DataQualityFramework(df_ventas, 'ventas_ecommerce')

# Evaluar todas las dimensiones
print("🚀 Iniciando evaluación de calidad...\n")

# 1. Completitud
dq.evaluar_completitud()

# 2. Unicidad
dq.evaluar_unicidad(columnas_clave=['venta_id'])

# 3. Validez
reglas_negocio = {
    'cantidad_positiva': 'cantidad > 0',
    'descuento_valido': 'descuento_pct >= 0 and descuento_pct <= 100',
    'total_positivo': 'total > 0'
}
dq.evaluar_validez(reglas_negocio)

# 4. Consistencia
relaciones = {
    'total_coherente': 'total >= (precio_unitario * cantidad * (1 - descuento_pct/100)) * 0.95',
}
dq.evaluar_consistencia(relaciones)

# 5. Exactitud
dq.evaluar_exactitud('precio_unitario', (5, 3000))

# 6. Actualidad
dq.evaluar_actualidad('fecha_venta', dias_maximos=365)

# Generar reporte completo
reporte = dq.generar_reporte_completo()

## 📋 Data Lineage - Trazabilidad de Datos

Implementación de un sistema básico de lineage:

### 🔬 **Great Expectations: Enterprise Data Quality**

**¿Cuándo usar Great Expectations vs Custom Framework?**

| Escenario | Recomendación |
|-----------|---------------|
| **Startup/SMB** | Custom Framework (simple, rápido) |
| **Enterprise** | Great Expectations (governance completo) |
| **Team < 5** | Custom |
| **Team > 10** | Great Expectations |
| **SQL-first team** | dbt tests |
| **Python-first team** | Great Expectations o Custom |

**Great Expectations Architecture:**

```
┌──────────────────────────────────────────────────────┐
│              DATA CONTEXT (GX Project)                │
│                                                       │
│  ┌─────────────────┐  ┌─────────────────┐          │
│  │  EXPECTATIONS   │  │   DATASOURCES   │          │
│  │  (Rules/Tests)  │  │   (Connections) │          │
│  │                 │  │                 │          │
│  │  • Suite 1      │  │  • Postgres     │          │
│  │  • Suite 2      │  │  • S3           │          │
│  │  • Suite 3      │  │  • Pandas DF    │          │
│  └─────────────────┘  └─────────────────┘          │
│                                                       │
│  ┌─────────────────┐  ┌─────────────────┐          │
│  │  CHECKPOINTS    │  │   DATA DOCS     │          │
│  │  (Validation    │  │   (Reports)     │          │
│  │   Workflows)    │  │                 │          │
│  └─────────────────┘  └─────────────────┘          │
└──────────────────────────────────────────────────────┘
```

**Setup:**

```python
import great_expectations as gx
from great_expectations.checkpoint import Checkpoint

# Initialize context
context = gx.get_context()

# Add datasource
datasource = context.sources.add_pandas("my_datasource")
data_asset = datasource.add_dataframe_asset(name="ventas_df")

# Create expectation suite
suite = context.add_expectation_suite("ventas_quality_suite")

# Get validator
batch_request = data_asset.build_batch_request(dataframe=df_ventas)
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="ventas_quality_suite"
)
```

**Expectations (200+ built-in):**

```python
# COMPLETITUD
validator.expect_column_values_to_not_be_null(
    column="venta_id",
    mostly=1.0  # 100% not null
)

validator.expect_column_values_to_not_be_null(
    column="email",
    mostly=0.8  # Al menos 80% not null
)

# UNICIDAD
validator.expect_column_values_to_be_unique(
    column="venta_id"
)

validator.expect_compound_columns_to_be_unique(
    column_list=["cliente_id", "fecha_venta", "producto_id"]
)

# VALIDEZ (Rango)
validator.expect_column_values_to_be_between(
    column="cantidad",
    min_value=1,
    max_value=1000
)

validator.expect_column_values_to_be_between(
    column="descuento_pct",
    min_value=0,
    max_value=100,
    mostly=0.99  # 99% dentro del rango
)

# VALIDEZ (Valores permitidos)
validator.expect_column_values_to_be_in_set(
    column="estado",
    value_set=["Completada", "Pendiente", "Cancelada"]
)

# VALIDEZ (Regex)
validator.expect_column_values_to_match_regex(
    column="email",
    regex=r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
)

# VALIDEZ (Tipo de dato)
validator.expect_column_values_to_be_of_type(
    column="total",
    type_="float"
)

# CONSISTENCIA (Cross-column)
validator.expect_column_pair_values_A_to_be_greater_than_B(
    column_A="fecha_entrega",
    column_B="fecha_pedido"
)

# EXACTITUD (Estadísticas)
validator.expect_column_mean_to_be_between(
    column="total",
    min_value=50,
    max_value=500
)

validator.expect_column_stdev_to_be_between(
    column="total",
    min_value=10,
    max_value=200
)

# ACTUALIDAD
validator.expect_column_max_to_be_between(
    column="fecha_venta",
    min_value=datetime.now() - timedelta(days=1),
    max_value=datetime.now()
)

# Save suite
validator.save_expectation_suite(discard_failed_expectations=False)
```

**Checkpoint (Automated Validation):**

```python
# Create checkpoint
checkpoint_config = {
    "name": "ventas_daily_checkpoint",
    "config_version": 1.0,
    "class_name": "Checkpoint",
    "validations": [
        {
            "batch_request": {
                "datasource_name": "my_datasource",
                "data_asset_name": "ventas_df"
            },
            "expectation_suite_name": "ventas_quality_suite"
        }
    ],
    "action_list": [
        {
            "name": "store_validation_result",
            "action": {"class_name": "StoreValidationResultAction"}
        },
        {
            "name": "update_data_docs",
            "action": {"class_name": "UpdateDataDocsAction"}
        },
        {
            "name": "send_slack_notification",
            "action": {
                "class_name": "SlackNotificationAction",
                "slack_webhook": "https://hooks.slack.com/...",
                "notify_on": "failure"
            }
        }
    ]
}

checkpoint = context.add_checkpoint(**checkpoint_config)

# Run checkpoint
result = checkpoint.run()

# Check result
if not result["success"]:
    print("❌ Validation failed!")
    for validation in result["run_results"].values():
        for expectation in validation["validation_result"]["results"]:
            if not expectation["success"]:
                print(f"  - {expectation['expectation_config']['expectation_type']}")
                print(f"    Observed: {expectation['result']['observed_value']}")
else:
    print("✅ All validations passed!")
```

**Data Docs (Auto-generated Reports):**

```python
# Build Data Docs
context.build_data_docs()

# Open in browser
context.open_data_docs()

# Output: Beautiful HTML report with:
# - Expectation suite overview
# - Validation results (pass/fail)
# - Data profiling statistics
# - Historical trends
```

**Integration con Airflow:**

```python
from airflow.operators.python import PythonOperator
import great_expectations as gx

def validate_with_ge(**context):
    gx_context = gx.get_context()
    
    # Get data
    df = context['ti'].xcom_pull(task_ids='extract_data')
    
    # Run checkpoint
    checkpoint = gx_context.get_checkpoint("ventas_daily_checkpoint")
    result = checkpoint.run(batch_request={"dataframe": df})
    
    # Fail task if validation fails
    if not result["success"]:
        raise AirflowException("Great Expectations validation failed")
    
    return result

validate_task = PythonOperator(
    task_id='validate_data_quality',
    python_callable=validate_with_ge
)
```

**Custom Expectations (Extensible):**

```python
from great_expectations.expectations.expectation import ColumnMapExpectation

class ExpectColumnValuesToBePrime(ColumnMapExpectation):
    """Custom: Validar números primos"""
    
    map_metric = "column_values.prime"
    
    @staticmethod
    def is_prime(n):
        if n < 2:
            return False
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0:
                return False
        return True
    
    def _validate(self, metrics, runtime_configuration, execution_engine):
        return self.is_prime(metrics.get("column_values"))

# Use
validator.expect_column_values_to_be_prime(column="id")
```

**Profiling (Auto-discovery):**

```python
# GX can auto-generate expectations
from great_expectations.profile.user_configurable_profiler import UserConfigurableProfiler

profiler = UserConfigurableProfiler(
    profile_dataset=validator,
    excluded_expectations=[
        "expect_column_values_to_be_unique"  # Exclude specific
    ]
)

suite = profiler.build_suite()
# Auto-generates ~20-30 expectations based on data profile
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
class DataLineageTracker:
    """
    Sistema para rastrear el linaje de datos
    """
    
    def __init__(self):
        self.linaje = []
        self.grafo = {}
    
    def registrar_transformacion(self, 
                                 origen: str, 
                                 destino: str, 
                                 transformacion: str,
                                 metadata: dict = None):
        """
        Registra una transformación de datos
        """
        evento = {
            'timestamp': datetime.now().isoformat(),
            'origen': origen,
            'destino': destino,
            'transformacion': transformacion,
            'metadata': metadata or {},
            'id': hashlib.md5(f"{origen}{destino}{datetime.now()}".encode()).hexdigest()[:8]
        }
        
        self.linaje.append(evento)
        
        # Construir grafo
        if origen not in self.grafo:
            self.grafo[origen] = []
        self.grafo[origen].append(destino)
        
        print(f"📝 Registrado: {origen} → {destino} ({transformacion})")
        return evento['id']
    
    def obtener_linaje_completo(self, entidad: str, direccion='downstream'):
        """
        Obtiene el linaje completo de una entidad
        """
        if direccion == 'downstream':
            # Hacia adelante (downstream)
            return self._traverse_downstream(entidad)
        else:
            # Hacia atrás (upstream)
            return self._traverse_upstream(entidad)
    
    def _traverse_downstream(self, nodo, visitados=None):
        if visitados is None:
            visitados = set()
        
        if nodo in visitados:
            return []
        
        visitados.add(nodo)
        camino = [nodo]
        
        if nodo in self.grafo:
            for hijo in self.grafo[nodo]:
                camino.extend(self._traverse_downstream(hijo, visitados))
        
        return camino
    
    def _traverse_upstream(self, nodo):
        # Invertir el grafo para upstream
        grafo_invertido = {}
        for origen, destinos in self.grafo.items():
            for destino in destinos:
                if destino not in grafo_invertido:
                    grafo_invertido[destino] = []
                grafo_invertido[destino].append(origen)
        
        return self._traverse_downstream(nodo, set())
    
    def visualizar_linaje(self):
        """
        Genera visualización simple del linaje
        """
        print("\n📊 MAPA DE LINAJE DE DATOS")
        print("="*50)
        
        for origen, destinos in self.grafo.items():
            print(f"\n🔹 {origen}")
            for destino in destinos:
                # Encontrar transformación
                trans = next((e for e in self.linaje 
                            if e['origen'] == origen and e['destino'] == destino), {})
                print(f"   └─→ {destino} [{trans.get('transformacion', 'N/A')}]")
    
    def exportar_linaje(self, ruta_salida: str):
        """
        Exporta linaje a archivo JSON
        """
        os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
        
        with open(ruta_salida, 'w', encoding='utf-8') as f:
            json.dump({
                'eventos': self.linaje,
                'grafo': self.grafo,
                'generado': datetime.now().isoformat()
            }, f, indent=2, ensure_ascii=False)
        
        print(f"\n💾 Linaje exportado a: {ruta_salida}")

# Ejemplo de uso
tracker = DataLineageTracker()

print("🔍 Simulando pipeline con lineage tracking:\n")

# Registrar transformaciones
tracker.registrar_transformacion(
    'raw_ventas_csv',
    'stage_ventas',
    'extraccion',
    {'filas': 1000, 'columnas': 9}
)

tracker.registrar_transformacion(
    'stage_ventas',
    'clean_ventas',
    'limpieza',
    {'nulos_removidos': 50, 'duplicados_removidos': 10}
)

tracker.registrar_transformacion(
    'clean_ventas',
    'agg_ventas_diarias',
    'agregacion',
    {'grupo_por': 'fecha', 'metricas': ['sum', 'count', 'avg']}
)

tracker.registrar_transformacion(
    'clean_ventas',
    'ventas_por_producto',
    'agregacion',
    {'grupo_por': 'producto'}
)

tracker.registrar_transformacion(
    'agg_ventas_diarias',
    'dashboard_ventas',
    'visualizacion',
    {'tipo': 'time_series'}
)

# Visualizar
tracker.visualizar_linaje()

# Exportar
tracker.exportar_linaje('../datasets/processed/data_lineage.json')

# Consultar linaje
print("\n🔎 Linaje downstream de 'clean_ventas':")
downstream = tracker.obtener_linaje_completo('clean_ventas', 'downstream')
print(f"   {' → '.join(downstream)}")

## 🎓 Resumen y Mejores Prácticas

### ✅ Principios Clave de Data Governance:

1. **Proactivo, no Reactivo**
   - Validaciones automáticas en cada etapa
   - Prevención vs corrección

2. **Cultura de Calidad**
   - Responsabilidad compartida
   - Data stewards claramente definidos

3. **Automatización**
   - Tests automatizados
   - Monitoreo continuo
   - Alertas proactivas

4. **Documentación**
   - Lineage completo
   - Metadata actualizado
   - Políticas claras

5. **Balance**
   - Gobernanza sin burocracia
   - Flexibilidad con control

### 🔜 Próximos Temas:

- Implementación de Data Catalogs
- Master Data Management (MDM)
- Privacy by Design y PII handling
- Compliance automation (GDPR, CCPA)
- Cost optimization y FinOps

---

**¡Has dominado los fundamentos de Data Governance!** 🏆

---

## 🧭 Navegación

**← Anterior:** [← README del Curso](../../README.md)

**Siguiente →:** [🏗️ Data Lakehouse con Parquet, Delta Lake e Iceberg (conceptos y práctica ligera) →](02_lakehouse_delta_iceberg.ipynb)

**📚 Índice de Nivel Senior:**
- [🏛️ Senior - 01. Data Governance y Calidad de Datos](01_data_governance_calidad.ipynb) ← 🔵 Estás aquí
- [🏗️ Data Lakehouse con Parquet, Delta Lake e Iceberg (conceptos y práctica ligera)](02_lakehouse_delta_iceberg.ipynb)
- [Apache Spark Streaming: Procesamiento en Tiempo Real](03_spark_streaming.ipynb)
- [🏛️ Arquitecturas Modernas de Datos: Lambda, Kappa, Delta y Data Mesh](04_arquitecturas_modernas.ipynb)
- [🤖 ML Pipelines y Feature Stores](05_ml_pipelines_feature_stores.ipynb)
- [💰 Cost Optimization y FinOps en la Nube](06_cost_optimization_finops.ipynb)
- [🔐 Seguridad, Compliance y Auditoría de Datos](07_seguridad_compliance.ipynb)
- [📊 Observabilidad y Linaje de Datos](08_observabilidad_linaje.ipynb)
- [🏆 Proyecto Integrador Senior 1: Plataforma de Datos Completa](09_proyecto_integrador_1.ipynb)
- [🌐 Proyecto Integrador Senior 2: Data Mesh Multi-Dominio con Feature Store](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
